In [1]:
import numpy as np
import random

# Define Rastrigin Function

In [2]:
def rastrigin(X):
    if isinstance(X[0], (int, float)):
        X = [[X[i]] for i in range(len(X))]
    
    val = []
    for xi in X:
        fx = 10 * len(xi) + sum(np.array(xi) ** 2 - 10 * np.cos(2 * np.pi * np.array(xi)))
        val.append(fx)
    return np.array(val)

# Generate Primary Target Vectors

In [3]:
def generate_target(num_variables,population_size,x_min,x_max):
    return np.random.randint(x_min,x_max,(population_size,num_variables))   #use uniform instead of randint

In [4]:
print(type(generate_target(2,10,-5,5)))

<class 'numpy.ndarray'>


In [5]:
rastrigin(generate_target(2,10,-5,5))

array([ 2.,  2., 26., 17., 18., 26., 20., 32.,  5., 16.])

In [6]:
target = list(generate_target(2,10,-5,5))
print(target)

[array([-1, -4]), array([-4, -3]), array([2, 2]), array([-5,  4]), array([-4, -4]), array([ 1, -2]), array([ 0, -4]), array([ 3, -5]), array([2, 4]), array([2, 2])]


In [7]:
x1,x2,x3= random.sample(target,3)
print(x1,x2,x3)
vector = x1 + .1*(x2-x3)
print(type(vector))

[-1 -4] [2 4] [-5  4]
<class 'numpy.ndarray'>


In [8]:
mutant =np.clip(np.array([random.sample(list(target), 3)[0] + 1 * (random.sample(list(target), 3)[1] - random.sample(list(target), 3)[2]) for _ in range(10)]),-5,5)
print({tuple(mutant[i]):rastrigin(mutant)[i] for i in range(10)})

{(5, 2): 29.0, (-5, -5): 50.0, (2, 2): 8.0, (-3, -5): 34.0, (-4, -3): 25.0, (5, -5): 50.0, (4, 5): 41.0}


# Differential Evolution

In [13]:
def diff_evol(num_iterations,population_size,num_variables,x_min,x_max,scale_factor,crossover_probability):
    target = generate_target(num_variables,population_size,x_min,x_max)
    trial = np.zeros_like(target)
    
    for i in range(num_iterations):
        mutant = np.clip(np.array([np.array(random.sample(list(target), 3)[0]) + scale_factor * (np.array(random.sample(list(target), 3)[1]) - np.array(random.sample(list(target), 3)[2])) for _ in range(population_size)]), x_min, x_max)
        
        for j in range(num_variables):
            I_rand = np.random.randint(0, num_variables)
            if np.random.uniform(0, 1) <= crossover_probability or j == I_rand:
                trial[j] = mutant[j]
            else:
                if np.random.uniform(0, 1) > crossover_probability and j != I_rand:
                    trial[j] = target[j]
                    continue             
        
        target_dict = {tuple(target[i]): rastrigin(target)[i] for i in range(population_size)}  # Fix: Corrected the dictionary creation
        trial_dict = {tuple(trial[i]): rastrigin(trial)[i] for i in range(population_size)}  # Fix: Corrected the dictionary creation
        
        target_dict.update(trial_dict)
        items = list(target_dict.items())
        items.sort(key=lambda x: x[1])
        new_target = [list(items[0]) for item in items[:population_size]]  # Fix: Extract the first element of each tuple
        
        target = np.array(new_target)
    
    return target

diff_evol(100, 50, 2, -5.12, 5.12, .5, .5)


/tmp/ipykernel_29766/3898687291.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  target = np.array(new_target)


TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'